# Zadanie klasyfikacji tekstu

Jak już wspomnieliśmy, skupimy się na prostym zadaniu klasyfikacji tekstu opartym na zbiorze danych **AG_NEWS**, które polega na klasyfikacji nagłówków wiadomości do jednej z 4 kategorii: Świat, Sport, Biznes i Nauka/Technologia.

## Zbiór danych

Ten zbiór danych jest wbudowany w moduł [`torchtext`](https://github.com/pytorch/text), dzięki czemu mamy do niego łatwy dostęp.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Tutaj `train_dataset` i `test_dataset` zawierają kolekcje, które zwracają pary etykiety (numer klasy) i tekstu odpowiednio, na przykład:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Więc wydrukujmy pierwsze 10 nowych nagłówków z naszego zestawu danych:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Ponieważ zestawy danych są iteratorami, jeśli chcemy używać danych wielokrotnie, musimy je przekonwertować na listę:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenizacja

Teraz musimy przekształcić tekst w **liczby**, które mogą być reprezentowane jako tensory. Jeśli chcemy reprezentację na poziomie słów, musimy zrobić dwie rzeczy:
* użyć **tokenizatora**, aby podzielić tekst na **tokeny**
* zbudować **słownik** tych tokenów.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Używając słownictwa, możemy łatwo zakodować ztokenizowany ciąg w zestaw liczb:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Reprezentacja tekstu za pomocą Bag of Words

Ponieważ słowa niosą znaczenie, czasami możemy zrozumieć sens tekstu, patrząc jedynie na pojedyncze słowa, niezależnie od ich kolejności w zdaniu. Na przykład, przy klasyfikacji wiadomości, słowa takie jak *pogoda*, *śnieg* prawdopodobnie wskazują na *prognozę pogody*, podczas gdy słowa takie jak *akcje*, *dolar* będą sugerować *wiadomości finansowe*.

Reprezentacja wektorowa **Bag of Words** (BoW) jest najczęściej używaną tradycyjną metodą reprezentacji wektorowej. Każde słowo jest powiązane z indeksem wektora, a element wektora zawiera liczbę wystąpień danego słowa w określonym dokumencie.

![Obraz pokazujący, jak reprezentacja wektorowa Bag of Words jest przechowywana w pamięci.](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png) 

> **Note**: Możesz również myśleć o BoW jako o sumie wszystkich wektorów zakodowanych w formie one-hot dla poszczególnych słów w tekście.

Poniżej znajduje się przykład generowania reprezentacji Bag of Words przy użyciu biblioteki Scikit Learn w Pythonie:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Aby obliczyć wektor bag-of-words z reprezentacji wektorowej naszego zbioru danych AG_NEWS, możemy użyć następującej funkcji:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Uwaga:** Tutaj używamy globalnej zmiennej `vocab_size`, aby określić domyślny rozmiar słownika. Ponieważ często rozmiar słownika jest dość duży, możemy ograniczyć jego rozmiar do najczęściej występujących słów. Spróbuj zmniejszyć wartość `vocab_size` i uruchomić poniższy kod, aby zobaczyć, jak wpływa to na dokładność. Powinieneś spodziewać się pewnego spadku dokładności, ale nie dramatycznego, w zamian za wyższą wydajność.


## Trenowanie klasyfikatora BoW

Teraz, gdy nauczyliśmy się tworzyć reprezentację Bag-of-Words dla naszego tekstu, przejdźmy do trenowania klasyfikatora na jej podstawie. Najpierw musimy przekształcić nasz zbiór danych do trenowania w taki sposób, aby wszystkie wektorowe reprezentacje pozycji zostały zamienione na reprezentację Bag-of-Words. Można to osiągnąć, przekazując funkcję `bowify` jako parametr `collate_fn` do standardowego torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Teraz zdefiniujmy prostą sieć neuronową klasyfikatora, która zawiera jedną warstwę liniową. Rozmiar wektora wejściowego jest równy `vocab_size`, a rozmiar wyjściowy odpowiada liczbie klas (4). Ponieważ rozwiązujemy zadanie klasyfikacji, końcową funkcją aktywacji jest `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Teraz zdefiniujemy standardową pętlę treningową PyTorch. Ponieważ nasz zbiór danych jest dość duży, na potrzeby nauczania będziemy trenować tylko przez jedną epokę, a czasami nawet krócej niż jedną epokę (określenie parametru `epoch_size` pozwala nam ograniczyć trening). Będziemy również raportować skumulowaną dokładność treningu podczas treningu; częstotliwość raportowania jest określana za pomocą parametru `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGramy, TriGramy i N-Gramy

Jednym z ograniczeń podejścia typu worek słów jest to, że niektóre słowa są częścią wyrażeń wielowyrazowych. Na przykład słowo „hot dog” ma zupełnie inne znaczenie niż słowa „hot” i „dog” w innych kontekstach. Jeśli zawsze reprezentujemy słowa „hot” i „dog” za pomocą tych samych wektorów, może to wprowadzać zamieszanie w naszym modelu.

Aby rozwiązać ten problem, w metodach klasyfikacji dokumentów często stosuje się **reprezentacje N-gramów**, gdzie częstotliwość każdego słowa, pary słów (bigramu) lub trójki słów (trigramu) jest przydatną cechą do trenowania klasyfikatorów. W reprezentacji bigramowej, na przykład, dodajemy wszystkie pary słów do słownika, oprócz oryginalnych słów.

Poniżej znajduje się przykład, jak wygenerować reprezentację bigramową w podejściu worek słów za pomocą Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Główną wadą podejścia N-gram jest to, że rozmiar słownika zaczyna rosnąć niezwykle szybko. W praktyce musimy połączyć reprezentację N-gram z technikami redukcji wymiarów, takimi jak *osadzenia* (*embeddings*), które omówimy w następnej jednostce.

Aby użyć reprezentacji N-gram w naszym zbiorze danych **AG News**, musimy zbudować specjalny słownik ngramów:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Możemy użyć tego samego kodu, co powyżej, aby wytrenować klasyfikator, jednak byłoby to bardzo nieefektywne pod względem pamięci. W następnej jednostce wytrenujemy klasyfikator bigramowy, korzystając z osadzeń.

> **Uwaga:** Możesz pozostawić tylko te ngramy, które występują w tekście więcej niż określoną liczbę razy. Dzięki temu rzadkie bigramy zostaną pominięte, co znacząco zmniejszy wymiarowość. Aby to zrobić, ustaw parametr `min_freq` na wyższą wartość i obserwuj zmianę długości słownika.


## Term Frequency Inverse Document Frequency TF-IDF

W reprezentacji BoW wystąpienia słów są równomiernie ważone, niezależnie od samego słowa. Jednak oczywiste jest, że częste słowa, takie jak *a*, *w* itd., są znacznie mniej istotne dla klasyfikacji niż terminy specjalistyczne. W rzeczywistości, w większości zadań NLP, niektóre słowa są bardziej istotne niż inne.

**TF-IDF** oznacza **częstość terminu–odwrotną częstość dokumentu**. Jest to wariacja torby słów, gdzie zamiast binarnej wartości 0/1 wskazującej na obecność słowa w dokumencie, używana jest wartość zmiennoprzecinkowa, która jest związana z częstością występowania słowa w korpusie.

Bardziej formalnie, waga $w_{ij}$ słowa $i$ w dokumencie $j$ jest definiowana jako:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
gdzie:
* $tf_{ij}$ to liczba wystąpień $i$ w $j$, czyli wartość BoW, którą widzieliśmy wcześniej
* $N$ to liczba dokumentów w kolekcji
* $df_i$ to liczba dokumentów zawierających słowo $i$ w całej kolekcji

Wartość TF-IDF $w_{ij}$ rośnie proporcjonalnie do liczby wystąpień słowa w dokumencie i jest korygowana o liczbę dokumentów w korpusie, które zawierają dane słowo, co pomaga uwzględnić fakt, że niektóre słowa pojawiają się częściej niż inne. Na przykład, jeśli słowo pojawia się *w każdym* dokumencie w kolekcji, $df_i=N$, a $w_{ij}=0$, i takie terminy byłyby całkowicie pomijane.

Możesz łatwo stworzyć wektoryzację TF-IDF tekstu za pomocą Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Podsumowanie

Mimo że reprezentacje TF-IDF nadają wagę częstotliwości różnym słowom, nie są w stanie oddać znaczenia ani kolejności. Jak powiedział słynny językoznawca J. R. Firth w 1935 roku: „Pełne znaczenie słowa zawsze zależy od kontekstu, a żadnego badania znaczenia poza kontekstem nie można traktować poważnie.”. W dalszej części kursu dowiemy się, jak uchwycić informacje kontekstowe z tekstu za pomocą modelowania języka.



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż staramy się zapewnić dokładność, prosimy mieć na uwadze, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji krytycznych zaleca się skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
